In [321]:
# !pip install gurobipy

In [322]:
import gurobipy as gb

In [323]:
import numpy as np
import pandas as pd

In [324]:
# data1 = data[(data["ORIGIN_AIRPORT"] == "LAX") | (data["DESTINATION_AIRPORT"] == "LAX")]
# data1


In [325]:
# data_for_day19 = data1[data1["DAY"] == 1]
# data_for_day19

In [326]:
# df1 = data1.drop(["YEAR", "ARRIVAL_DELAY", "MONTH", "CANCELLATION_REASON", "ARRIVAL_TIME", "DEPARTURE_TIME", "TAXI_OUT", "WHEELS_OFF", "ELAPSED_TIME", "AIR_TIME", "WHEELS_ON", "TAXI_IN", "ARRIVAL_TIME", "DIVERTED", "CANCELLED", "AIRLINE_DELAY", "AIR_SYSTEM_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY", "WEATHER_DELAY"], axis =1)
# df = df1[df1["DAY"] == 19].copy()
# df
# output_filename = 'my_dataframe.csv'

# Save the DataFrame to a CSV file
# index=False prevents pandas from writing the DataFrame index as a column in the CSV
# df.to_csv(output_filename, index=False)


In [327]:
df = pd.read_csv("E:/Download Till 27-07-2024/Downloads/my_dataframe.csv")
df
df1 = df[(df["DESTINATION_AIRPORT"] == "LAX" )]
df1

,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL
8,19,4,WN,4036,N655WN,SMF,LAX,540,-1.0,85.0,373,705
11,19,4,AA,302,N634AA,MCO,LAX,550,0.0,340.0,2218,830
12,19,4,OO,6465,N782SK,SLC,LAX,550,-4.0,125.0,590,655
15,19,4,OO,5373,N919SW,SBA,LAX,555,5.0,56.0,89,651
16,19,4,AA,171,N788AA,JFK,LAX,600,-7.0,382.0,2475,922
...,...,...,...,...,...,...,...,...,...,...,...,...
1183,19,4,DL,1298,N624AG,KOA,LAX,2247,-6.0,323.0,2504,610
1189,19,4,DL,1840,N548US,LIH,LAX,2259,-4.0,336.0,2615,635
1191,19,4,AA,14,N5DEAA,OGG,LAX,2300,25.0,304.0,2486,604
1196,19,4,DL,2116,N547US,OGG,LAX,2315,13.0,320.0,2486,635


In [328]:
df2 = df[df["ORIGIN_AIRPORT"] == "LAX"]
df2

,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL
0,19,4,AA,2400,N5ECAA,LAX,DFW,5,5.0,168.0,1235,453
1,19,4,AA,258,N3CCAA,LAX,MIA,20,-2.0,284.0,2342,804
2,19,4,US,1866,N521UW,LAX,CLT,35,-5.0,269.0,2125,804
3,19,4,DL,892,N589NW,LAX,MSP,40,0.0,211.0,1535,611
4,19,4,UA,1500,N76517,LAX,IAH,46,-2.0,188.0,1379,554
...,...,...,...,...,...,...,...,...,...,...,...,...
1204,19,4,B6,688,N784JB,LAX,BOS,2349,0.0,320.0,2611,809
1205,19,4,DL,1254,N822DN,LAX,ATL,2350,5.0,240.0,1947,650
1206,19,4,VX,170,N851VA,LAX,EWR,2350,-4.0,300.0,2454,750
1207,19,4,UA,1557,N36472,LAX,EWR,2353,44.0,306.0,2454,759


In [329]:
# !pip install pulp

In [330]:
# Step 1: Convert HHMM to minutes since midnight
df['DEPARTURE_SLOT'] = df['SCHEDULED_DEPARTURE'] // 100 * 60 + df['SCHEDULED_DEPARTURE'] % 100

# Step 2: Bucket each flight into a 15-minute time slot
df['DEPARTURE_BUCKET'] = df['DEPARTURE_SLOT'] // 15

# Optional: Convert bucket back into readable time for checking
df['BUCKET_HOUR'] = (df['DEPARTURE_BUCKET'] * 15) // 60
df['BUCKET_MINUTE'] = (df['DEPARTURE_BUCKET'] * 15) % 60
df['BUCKET_TIME'] = df['BUCKET_HOUR'].astype(str).str.zfill(2) + ':' + df['BUCKET_MINUTE'].astype(str).str.zfill(2)
df['ORIGINAL_BUCKET'] = df['DEPARTURE_SLOT'] // 15

df['ARRIVAL_SLOT'] = df['SCHEDULED_ARRIVAL'] // 100 * 60 + df['SCHEDULED_ARRIVAL'] % 100
df['ORIGINAL_BUCKET_ARR'] = df['ARRIVAL_SLOT'] // 15
# df_turnaround = df.copy() # Work on a copy
# Handle NaNs in ARRIVAL_SLOT if necessary - flights might not have arrived in this dataset
# If a flight doesn't arrive in this dataset, it cannot be the incoming leg of a turnaround pair here.
df = df.dropna(subset=['TAIL_NUMBER', 'ARRIVAL_SLOT', 'DEPARTURE_SLOT'])
df

,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DEPARTURE_SLOT,DEPARTURE_BUCKET,BUCKET_HOUR,BUCKET_MINUTE,BUCKET_TIME,ORIGINAL_BUCKET,ARRIVAL_SLOT,ORIGINAL_BUCKET_ARR
0,19,4,AA,2400,N5ECAA,LAX,DFW,5,5.0,168.0,1235,453,5,0,0,0,00:00,0,293,19
1,19,4,AA,258,N3CCAA,LAX,MIA,20,-2.0,284.0,2342,804,20,1,0,15,00:15,1,484,32
2,19,4,US,1866,N521UW,LAX,CLT,35,-5.0,269.0,2125,804,35,2,0,30,00:30,2,484,32
3,19,4,DL,892,N589NW,LAX,MSP,40,0.0,211.0,1535,611,40,2,0,30,00:30,2,371,24
4,19,4,UA,1500,N76517,LAX,IAH,46,-2.0,188.0,1379,554,46,3,0,45,00:45,3,354,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204,19,4,B6,688,N784JB,LAX,BOS,2349,0.0,320.0,2611,809,1429,95,23,45,23:45,95,489,32
1205,19,4,DL,1254,N822DN,LAX,ATL,2350,5.0,240.0,1947,650,1430,95,23,45,23:45,95,410,27
1206,19,4,VX,170,N851VA,LAX,EWR,2350,-4.0,300.0,2454,750,1430,95,23,45,23:45,95,470,31
1207,19,4,UA,1557,N36472,LAX,EWR,2353,44.0,306.0,2454,759,1433,95,23,45,23:45,95,479,31


##MODEL

In [331]:
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary

In [332]:
# Build Optimization Model
model = LpProblem("Flight_Rescheduling", LpMinimize)

In [333]:
# set(df['ORIGINAL_BUCKET'])
# {1,2,3,4} + {5,6,7}
# sorted(set(df['ORIGINAL_BUCKET']))

##PARAMETERS

In [334]:
# Parameters
buckets = sorted(set(df['ORIGINAL_BUCKET'])) + [bucket+1 for bucket in set(df['ORIGINAL_BUCKET'])] + [bucket-1 for bucket in set(df['ORIGINAL_BUCKET'])]
valid_start_bucket = 0
buckets = sorted(b for b in set(buckets) if b >= valid_start_bucket)
bucket_capacity = 20  # Max number of flights per bucket
flight_indices = df.index.tolist()


##DECISION VARIABLE

In [335]:
# Decision Variables: x[i][b] = 1 if flight i (identified by its DataFrame index) is scheduled in bucket b

# No need to create a new indexed DataFrame if we use the default index
# df_indexed = df.set_index('TAIL_NUMBE') # No longer needed in this approach

x = {
    (i, b): LpVariable(f"x_{i}_{b}", cat=LpBinary)
    for i in flight_indices  # Iterate over the DataFrame's unique indices (0, 1, 2, ...)
    for b in [df.loc[i, 'ORIGINAL_BUCKET'] - 1, # Access using the DataFrame index 'i'
              df.loc[i, 'ORIGINAL_BUCKET'],     # Access using the DataFrame index 'i'
              df.loc[i, 'ORIGINAL_BUCKET'] + 1] # Access using the DataFrame index 'i'
    if b in buckets # This check is now comparing a single integer bucket number 'b' with the 'buckets' list
}



In [336]:
#Another Variable

positive_deviation_from_predicted = LpVariable.dicts(
    "pos_dev_pred", flight_indices, lowBound=0, cat='Continuous'
)


# d_plus = LpVariable.dicts("d_plus", flight_indices, lowBound=0, cat='Continuous')
# d_minus = LpVariable.dicts("d_minus", flight_indices, lowBound=0, cat='Continuous')

In [337]:
df['DEPARTURE_DELAY'] = df['DEPARTURE_DELAY'].fillna(0)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DAY                  1209 non-null   int64  
 1   DAY_OF_WEEK          1209 non-null   int64  
 2   AIRLINE              1209 non-null   object 
 3   FLIGHT_NUMBER        1209 non-null   int64  
 4   TAIL_NUMBER          1209 non-null   object 
 5   ORIGIN_AIRPORT       1209 non-null   object 
 6   DESTINATION_AIRPORT  1209 non-null   object 
 7   SCHEDULED_DEPARTURE  1209 non-null   int64  
 8   DEPARTURE_DELAY      1209 non-null   float64
 9   SCHEDULED_TIME       1209 non-null   float64
 10  DISTANCE             1209 non-null   int64  
 11  SCHEDULED_ARRIVAL    1209 non-null   int64  
 12  DEPARTURE_SLOT       1209 non-null   int64  
 13  DEPARTURE_BUCKET     1209 non-null   int64  
 14  BUCKET_HOUR          1209 non-null   int64  
 15  BUCKET_MINUTE        1209 non-null   i

##OBJECTIVE

In [338]:
# Objective: Minimize total weighted delay

# Access DEPARTURE_DELAY using the DataFrame's index 'i'
# model += lpSum(x[i, b] * df.loc[i, 'DEPARTURE_DELAY'].item() for (i, b) in x)


In [339]:
# Remove or comment out the old objective:
# model += lpSum(x[i, b] * df.loc[i, 'DEPARTURE_DELAY'].item() for (i, b) in x)

# Keep the deviation objective (after defining d_plus and d_minus, and the deviation constraints):
# Example Objective: Minimize sum of deviations from original bucket (in terms of buckets)

# model += lpSum(d_plus[i] + d_minus[i] for i in flight_indices)

# Set the new objective: Minimize the total positive deviation from the predicted departure times
model += lpSum(positive_deviation_from_predicted[i] for i in flight_indices), "Minimize_Total_Positive_Predicted_Deviation"



##CONSTRAINTS

**Deviation Constraint**

In [340]:
# Assuming 'PREDICTED_DEPARTURE_DELAY' column (in minutes) is already in df

# --- Remove old deviation variables and constraints ---
# Remove or comment out:
# d_plus = LpVariable.dicts(...)
# d_minus = LpVariable.dicts(...)
# for i in flight_indices:
#     model += assigned_bucket_i - original_bucket_i == d_plus[i] - d_minus[i] , ...

# --- Define New Deviation Variables ---
# Define a variable for positive deviation from the PREDICTED departure time

# --- Add Constraints for Deviation from Predicted Time ---
for i in flight_indices:
    # Calculate the assigned departure time in minutes since midnight
    # (This uses your existing x variables and the logic for assigned_bucket_i, converted to minutes)
    assigned_departure_minutes_i = lpSum(
        (b * 15) * x[i, b]
        for b in [df.loc[i, 'ORIGINAL_BUCKET'] - 1,
                  df.loc[i, 'ORIGINAL_BUCKET'],
                  df.loc[i, 'ORIGINAL_BUCKET'] + 1]
        if (i, b) in x
    )

    # Calculate the predicted departure time in minutes since midnight
    original_scheduled_minutes_i = df.loc[i, 'SCHEDULED_DEPARTURE'] // 100 * 60 + df.loc[i, 'SCHEDULED_DEPARTURE'] % 100

    # IMPORTANT: Make sure 'PREDICTED_DEPARTURE_DELAY' is in minutes
    predicted_departure_minutes_i = original_scheduled_minutes_i + df.loc[i, 'DEPARTURE_DELAY']

    # Constraint: Assigned time minus Predicted time <= positive deviation
    # This means if assigned > predicted, pos_dev_pred captures the difference.
    # If assigned <= predicted, pos_dev_pred can be 0 (and will be driven to 0 by objective).
    model += assigned_departure_minutes_i - predicted_departure_minutes_i <= positive_deviation_from_predicted[i], \
             f"Predicted_Positive_Deviation_{i}"

# --- Set the New Objective Function ---
# Remove or comment out your old objective:
# model += lpSum(...)



In [341]:
# for i in flight_indices:
#         original_bucket_i = df.loc[i, 'ORIGINAL_BUCKET']
#         assigned_bucket_i = lpSum(b * x[i, b] for b in [original_bucket_i - 1, original_bucket_i, original_bucket_i + 1] if (i, b) in x)
#         # assigned_bucket - original_bucket = d_plus - d_minus
#         model += assigned_bucket_i - original_bucket_i == d_plus[i] - d_minus[i] , f"Deviation_{i}"


**Capacity Consraint**

1). *1 flight in 1 bucket*


2). *total flight in a bucket*




In [342]:
# %%
# Example of adding a capacity constraint (requires further implementation)
# For each bucket b, the sum of x[i, b] over all flights i should be <= bucket_capacity
for b in buckets:
    model += lpSum(x[i, b] for i in flight_indices if (i, b) in x) <= bucket_capacity , f"Bucket_Capacity_{b}"

# You will also need constraints to ensure each flight is assigned to exactly one bucket:
for i in flight_indices:
    model += lpSum(x[i, b] for b in [df.loc[i, 'ORIGINAL_BUCKET'] - 1,
                                      df.loc[i, 'ORIGINAL_BUCKET'],
                                      df.loc[i, 'ORIGINAL_BUCKET'] + 1]
                                      if b in buckets) == 1 , f"Assign_Flight_{i}"

**Turnaround Time**

In [343]:
# --- Step 1: Data Preparation for Turnaround ---
# Need the full dataset for Day 19 (and potentially parts of Day 18/20 if turnarounds span days)
# Let's use the data for Day 19 that you already filtered: df

# Ensure necessary columns are available and have appropriate types
# You'll need 'TAIL_NUMBE', 'SCHEDULED_ARRIVAL', 'SCHEDULED_DEPARTURE'
# Convert times to consistent units (e.g., minutes since midnight) if not already done
# (You've already done this for departure slots, you'll need it for arrivals too)

# Example: Convert SCHEDULED_ARRIVAL to minutes since midnight if it's in HHMM format
# Make sure to handle potential NaNs in arrival times if they are needed for turnarounds



df_turnaround = df.copy()

# Identify sequences of flights for each aircraft
# Sort by TAIL_NUMBE and then by time (arrival or departure, depending on which is first in the sequence)
df_turnaround = df_turnaround.sort_values(by=['TAIL_NUMBER', 'ARRIVAL_SLOT']) # Or departure slot, need to be careful about sequencing

turnaround_pairs = []
# Group by aircraft and look for arrival-departure sequences
for tail_num, group in df_turnaround.groupby('TAIL_NUMBER'):  #group is dataframe having rows containing same dataframe.
    # Sort flights for this aircraft by scheduled time
    # This logic needs to be robust - it's not always simple arrival then departure.
    # A flight arriving at LAX must precede a flight departing LAX by the same tail number.
    # This often requires looking at origin/destination too.
    inbound_flights = group[group['DESTINATION_AIRPORT'] == 'LAX'].sort_values('ARRIVAL_SLOT')
    outbound_flights = group[group['ORIGIN_AIRPORT'] == 'LAX'].sort_values('DEPARTURE_SLOT')

    # This next part is complex: finding the *specific* arrival that enables a *specific* departure.
    # You need to match arrivals and departures based on the sequence in time.
    # A simple approach (potentially too simplistic): find the earliest departing flight
    # by this tail number that occurs after an arriving flight.
    # A more rigorous approach involves pairing consecutive flights.

    # --- Simplified pairing logic (conceptual - may need refinement) ---
    # This assumes a simple sequence of Arrive -> Depart for each aircraft within the day.
    # A real implementation would need to handle multiple legs, maintenance, etc.
    # if not inbound_flights.empty and not outbound_flights.empty:
    #     # Example: find the first departure after the first arrival
    #     # This is NOT a general solution. A real solution needs to match consecutive legs.
    #     # For demonstration, let's assume we can identify *some* pairs.
    #     # A proper approach would iterate through sorted flights and find the next departure after an arrival.
    #     # For example:
    #     current_arrival_time = -np.inf # Initialize
    #     for idx, row in group.sort_values(['ARRIVAL_SLOT', 'DEPARTURE_SLOT']).iterrows():
    #          if row['DESTINATION_AIRPORT'] == 'LAX':
    #              current_arrival_time = row['ARRIVAL_SLOT']
    #          elif row['ORIGIN_AIRPORT'] == 'LAX' and row['DEPARTURE_SLOT'] > current_arrival_time:
    #              # This is a potential turnaround pair: previous arrival enables this departure
    #              # You need the index of the *arriving* flight and the *departing* flight
    #              # This simplified loop doesn't easily give you the *index* of the arriving flight row
    #              # A better approach iterates through arrivals and finds the next valid departure
    #              pass # Need more sophisticated pairing logic

    # --- Alternative pairing logic (Conceptual and more complex) ---
    # You need to iterate through the flights of a tail number, ordered by time,
    # and identify when an arrival at LAX is followed by a departure from LAX.
    # This requires careful indexing and state tracking.

    # Let's assume for now you have a list of tuples: (arriving_flight_index, departing_flight_index)
    # where both indices are from the original df and use the same TAIL_NUMBE
    # For simplicity in this example, let's manually create a dummy pair if possible for illustration
    # Replace this with your actual pairing logic:
    if len(group) >= 2:
         # Find an arriving flight and a subsequent departing flight
         inbound = group[group['DESTINATION_AIRPORT'] == 'LAX'].iloc[0] if not group[group['DESTINATION_AIRPORT'] == 'LAX'].empty else None
         outbound = group[group['ORIGIN_AIRPORT'] == 'LAX'].iloc[0] if not group[group['ORIGIN_AIRPORT'] == 'LAX'].empty else None

         if inbound is not None and outbound is not None and outbound['DEPARTURE_SLOT'] > inbound['ARRIVAL_SLOT']:
              # Add this pair (using their original DataFrame indices) to the list
              turnaround_pairs.append((inbound.name, outbound.name)) # Use .name for the original index




In [344]:
len(turnaround_pairs)*2

656

In [345]:
# turnaround_pairs

In [346]:
# --- Step 2: Define Minimum Turnaround Time ---
# Minimum turnaround time in minutes. Example: 60 minutes
min_turnaround_minutes = 30
min_turnaround_buckets = int(np.ceil(min_turnaround_minutes / 15)) # Convert to number of 15-min buckets

# --- Step 3: Add Turnaround Constraints to PuLP Model ---

# For each identified turnaround pair:
# (arriving_flight_index, departing_flight_index)
# Constraint: departing_flight's_assigned_bucket >= arriving_flight's_arrival_bucket + min_turnaround_buckets

# Your decision variables 'x[i, b]' currently assign departing flights (index 'i') to buckets 'b'.
# You need the arrival bucket of the incoming flight (arriving_flight_index).
# Since you are not rescheduling arrivals, the arrival bucket is fixed based on the scheduled arrival time.

for arriving_idx, departing_idx in turnaround_pairs:
    # Get the scheduled arrival time in buckets for the arriving flight
    # Assuming ARRIVAL_SLOT is in minutes since midnight
    scheduled_arrival_minutes = df.loc[arriving_idx, 'ARRIVAL_SLOT'] # Assuming ARRIVAL_SLOT is already calculated and available in df
    scheduled_arrival_bucket = int(scheduled_arrival_minutes // 15)

    # The departure bucket for the departing flight (departing_idx) must be at least
    # the arrival bucket of the arriving flight + minimum turnaround buckets.

    # The left side of the constraint is the assigned departure bucket for departing_idx.
    # This is represented by the sum of (bucket_number * x[departing_idx, b]) over possible buckets b.
    # Only add the constraint if the departing flight index is actually included in your model's variables 'x'
    if departing_idx in flight_indices: # Check if this flight was included in the model variables
        model += lpSum(b * x[departing_idx, b] for b in [df.loc[departing_idx, 'ORIGINAL_BUCKET'] - 1,
                                                       df.loc[departing_idx, 'ORIGINAL_BUCKET'],
                                                       df.loc[departing_idx, 'ORIGINAL_BUCKET'] + 1]
                                                       if (departing_idx, b) in x) \
                 >= scheduled_arrival_bucket + min_turnaround_buckets , \
                 f"Turnaround_{arriving_idx}_{departing_idx}"

# Note: This constraint formulation assumes that the arrival time is fixed.
# If arrivals were also being rescheduled, the constraint would need to link
# arrival decision variables to departure decision variables.

# This code is illustrative. The most complex part is accurately identifying `turnaround_pairs`.
# You need to write robust code to group by `TAIL_NUMBE` and find consecutive
# LAX arrival -> LAX departure sequences using the time information.

In [347]:
print("Solving the optimization model...")
model.solve()
print("Solving complete.")

Solving the optimization model...
Solving complete.


In [358]:
import pandas as pd
from pulp import LpProblem, LpMinimize, value, LpVariable, lpSum, LpBinary, LpStatus, LpStatusOptimal

In [359]:
print("Status:", model.status)
print("Status (string):", LpStatus[model.status])


Status: 1
Status (string): Optimal


In [360]:
print("Optimal Objective Value:", value(model.objective))


Optimal Objective Value: 361.0


In [361]:
print("\nSolution:")


Solution:


In [362]:
!pip install gurobipy


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [363]:
# # After defining the model
# model.solve(gb.Gurobi())
# # ... check status and objective ...

In [364]:
# After defining the model
from pulp import GUROBI # Import the Gurobi solver class from pulp

# Create an instance of the GUROBI solver
solver = GUROBI()

# Solve the model using the GUROBI solver
model.solve(solver)
# ... check status and objective ...

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 3450U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2828 rows, 4835 columns and 13069 nonzeros
Model fingerprint: 0x36e5c36b
Variable types: 1209 continuous, 3626 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 1699 rows and 1586 columns
Presolve time: 0.05s
Presolved: 1129 rows, 3249 columns, 6497 nonzeros
Variable types: 0 continuous, 3249 integer (3241 binary)

Root relaxation: objective 3.610000e+02, 1142 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0            

1

In [366]:
# import pandas as pd
# from pulp import LpStatus, value # Import value and LpStatus if you haven't already

# # --- Assuming 'model' has been solved successfully ---

# # Check if the model found an optimal solution
# if model.status == LpStatusOptimal:
#     print("\nOptimal Solution Found. Extracting Schedule...")

#     # Create new columns in your DataFrame to store the assigned schedule
#     # Initialize with a default value (e.g., None or a placeholder)
#     df['ASSIGNED_BUCKET'] = None
#     df['ASSIGNED_SLOT_MINUTES'] = None
#     df['ASSIGNED_TIME_HHMM'] = None
#     df['ASSIGNED_TIME_RANGE'] = None # Optional: To show the 15-min range

#     # Iterate through each flight that was included in the optimization model
#     for i in flight_indices:
#         assigned_bucket = None
#         # Check the value of the decision variable for each possible bucket for this flight
#         # The range of buckets to check must match how 'x' was defined for this flight
#         possible_buckets_for_flight = [df.loc[i, 'ORIGINAL_BUCKET'] - 1,
#                                        df.loc[i, 'ORIGINAL_BUCKET'],
#                                        df.loc[i, 'ORIGINAL_BUCKET'] + 1]

#         for b in possible_buckets_for_flight:
#              # Ensure the variable (i, b) actually exists in the model's variables
#              if (i, b) in model.variablesDict(): # Access model.variablesDict() to check if variable exists
#                 # Check if this variable's value is close to 1 (it's binary)
#                 # Use a small tolerance (like 0.5) for checking binary values due to potential floating-point precision
#                 if value(model.variablesDict()[(i, b)]) > 0.5:
#                     assigned_bucket = b
#                     break # Found the assigned bucket for this flight, no need to check others

#         # Store the assigned bucket in the DataFrame for this flight's original index
#         if assigned_bucket is not None:
#             df.loc[i, 'ASSIGNED_BUCKET'] = assigned_bucket

#             # Calculate Assigned Time in Minutes and HH:MM format
#             assigned_minutes_since_midnight = assigned_bucket * 15
#             df.loc[i, 'ASSIGNED_SLOT_MINUTES'] = assigned_minutes_since_midnight

#             hours = int(assigned_minutes_since_midnight // 60)
#             minutes = int(assigned_minutes_since_midnight % 60)
#             df.loc[i, 'ASSIGNED_TIME_HHMM'] = f"{hours:02d}:{minutes:02d}"

#             # Calculate the end time of the bucket for the range (Optional)
#             assigned_slot_end_minutes = assigned_minutes_since_midnight + 15
#             end_hours = int(assigned_slot_end_minutes // 60)
#             end_minutes = int(assigned_slot_end_minutes % 60)
#             df.loc[i, 'ASSIGNED_TIME_RANGE'] = f"{hours:02d}:{minutes:02d} - {end_hours:02d}:{end_minutes:02d}"

#     # --- Optional: Extract Deviation Values ---
#     # If you still have deviation variables (like positive_deviation_from_predicted)
#     # you can extract their values too
# # --- Optional: Extract Deviation Values ---
#     # If you included deviation variables (like positive_deviation_from_predicted) in your model,
#     # extract their values.
#     # Check if the variable dictionary contains variables starting with the prefix 'pos_dev_pred'
#     # as a way to determine if these variables exist in the model
#     deviation_variables_exist = any(var_name.startswith('pos_dev_pred_') for var_name in model.variablesDict())


#     if deviation_variables_exist:
#          # *** FIX IS HERE: Explicitly create the column before using it ***
#          # Initialize the column with a default value (e.g., 0.0 for continuous)
#          df['POSITIVE_PREDICTED_DEVIATION'] = 0.0

#          for i in flight_indices:
#               # Construct the variable name as it was defined in LpVariable.dicts
#               var_name = f"pos_dev_pred_{i}"
#               if var_name in model.variablesDict():
#                    # Use df.loc for setting values based on index
#                    df.loc[i, 'POSITIVE_PREDICTED_DEVIATION'] = value(model.variablesDict()[var_name])
#               # Else: If a flight index i was not in flight_indices used for the variable dict,
#               # the default value (0.0) will remain, which is usually fine.

#     print("\nDataFrame updated with Assigned Schedule:")
#     # Display relevant columns to see the result
#     print(df[['TAIL_NUMBER', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE',
#               'ORIGINAL_BUCKET', 'BUCKET_TIME', 'ASSIGNED_BUCKET', 'ASSIGNED_TIME_HHMM',
#               'ASSIGNED_TIME_RANGE', 'DEPARTURE_DELAY', 'POSITIVE_PREDICTED_DEVIATION']].head()) # Adjust columns as needed

#     # --- Save the updated DataFrame to a new CSV file ---
#     output_csv_path = 'rescheduled_flights_with_times.csv'
#     df.to_csv(output_csv_path, index=False)
#     print(f"\nRescheduled data saved to {output_csv_path}")

# else:
#     print("Model was not solved to optimality. Cannot extract solution.")
#     print("Status:", LpStatus[model.status])


Optimal Solution Found. Extracting Schedule...

DataFrame updated with Assigned Schedule:
  TAIL_NUMBER  FLIGHT_NUMBER ORIGIN_AIRPORT DESTINATION_AIRPORT  \
0      N5ECAA           2400            LAX                 DFW   
1      N3CCAA            258            LAX                 MIA   
2      N521UW           1866            LAX                 CLT   
3      N589NW            892            LAX                 MSP   
4      N76517           1500            LAX                 IAH   

   SCHEDULED_DEPARTURE  ORIGINAL_BUCKET BUCKET_TIME ASSIGNED_BUCKET  \
0                    5                0       00:00            None   
1                   20                1       00:15            None   
2                   35                2       00:30            None   
3                   40                2       00:30            None   
4                   46                3       00:45            None   

  ASSIGNED_TIME_HHMM ASSIGNED_TIME_RANGE  DEPARTURE_DELAY  \
0               No

In [355]:
# if model.status == 3: # Check if status is infeasible
#         print("Model is infeasible. Computing IIS...")
#         model.computeIIS() # This computes the Irreducible Infeasible Subsystem
#         model.write("infeasible_model.ilp") # Write the model with IIS annotations to a file
#         print("IIS written to infeasible_model.ilp")

In [356]:
# Yes, aircraft availability, crew scheduling, and airport slot regulation can all be incorporated into an optimization model like the one you are building, but they add significant complexity. Your current model focuses on airport slot regulation (the bucket capacity constraints) and a simplified form of aircraft availability (the turnaround constraint).

# Here's a breakdown of how you would typically approach incorporating these factors and the challenges involved:

# Aircraft Availability (Beyond Simple Turnaround):

# Current Model: Your current turnaround constraint (departing_flight's_assigned_bucket >= arriving_flight's_arrival_bucket + min_turnaround_buckets) is a basic form of aircraft availability, ensuring an aircraft is physically present after its previous flight before departing again from the same airport.
# More Complex Availability: A full aircraft routing and scheduling model would track each aircraft's position and status over time. This requires knowing the full sequence of flights for each aircraft over a longer period (potentially multiple days), including flights to/from other airports.
# Decision Variables: You might need additional decision variables to represent which physical aircraft (tail number) is assigned to which flight.
# Constraints: Constraints would ensure:
# Each flight is assigned exactly one available aircraft.
# An aircraft's sequence of assigned flights is feasible (arrival at an airport followed by departure from that same airport, respecting minimum turnaround times at all airports, not just LAX).
# Aircraft maintenance requirements are met.
# Crew Scheduling:

# Complexity: Crew scheduling is one of the most complex aspects of airline operations. It involves ensuring qualified crew members (pilots, flight attendants) are assigned to flights, adhering to strict regulations on duty periods, rest times, qualifications for specific aircraft types or routes, and base locations.
# Decision Variables: You would need variables for assigning specific crew members or crew "pairings" (sequences of flights for a crew) to flights.
# Constraints: Constraints would enforce all regulatory requirements (FAA, EASA, etc.), collective bargaining agreements, and company policies regarding:
# Duty period limits.
# Required rest periods between duties.
# Crew qualifications for aircraft types and routes.
# Crew base requirements (crews must end up where they are based).
# Minimum crew numbers for each flight type.
# Integration with Flight Schedule: Crew scheduling is heavily dependent on the flight schedule. Changes in flight times or cancellations directly impact crew feasibility.
#  This typically means solving flight scheduling and crew scheduling either sequentially (iteratively) or simultaneously (which is very complex).
# Airport Slot Regulation:

# Current Model: Your bucket capacity constraint (lpSum(x[i, b] for i in flight_indices if (i, b) in x) <= bucket_capacity) directly addresses one form of airport slot regulation – a limit on the number of flights departing (or arriving, if you added arrival slots) within a specific time interval (your 15-minute buckets).
# Other Regulations: Airport slot regulations can be more nuanced, sometimes specifying total daily/hourly movements, peak hour limits, or specific slot allocations assigned to airlines. Your current model simplifies this to a general capacity constraint.
# Integration: This constraint directly links the flight assignment decision variables (x[i, b]) to the airport capacity limits.
# Integrating these into a single model:

# While theoretically possible to build one massive optimization model covering all three aspects, it would be extremely large and computationally challenging to solve.

# Large-Scale Models: Airlines use sophisticated optimization software (often specialized commercial solvers and platforms) to tackle these integrated problems.
# Decomposition: More commonly, the problem is broken down into smaller, more manageable subproblems that are solved sequentially or iteratively. For example, first optimize the flight schedule respecting airport slots and minimum turnarounds, then take that schedule and optimize crew assignments, then check if any crew issues require schedule adjustments and iterate.
# Data Requirements: Integrating these requires vast amounts of detailed data on aircraft, crews, regulations, and the entire flight network, not just flights at a single airport on a single day.
# In summary:

# Your current model is a good starting point focusing on rescheduling flights within a limited timeframe and considering simplified airport capacity and aircraft turnaround. Expanding it to include full aircraft routing and comprehensive crew scheduling is a significant undertaking that moves towards the complexity of real-world airline operations planning. You can
# represent these with decision variables and constraints in a linear programming framework, but the scale and data requirements increase dramatically.